In [17]:
import os
import pandas as pd
import numpy as np
from copy import copy
from tqdm import tqdm
from sklearn.base import BaseEstimator
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
#import cover.cpp ToDo

In [1]:
def write(name_dataset, X, y):
    """
    Запись обучающей выборки в отдельный файл
    """
    with open(name_dataset, 'w') as f:
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                f.write(str(int(X[i][j])))
                f.write(' ')
            f.write(str(y[i]) + ' ')
            f.write('\n')

## Boosting  с минимизацией перебором по покрытиям (ToDo: Python оболочка для вызова C++ методов)

In [4]:
class BoostingRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, n_estimators=100):
        self.n_estimators = n_estimators

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y

        self.estimators_ = []
        current_estimate = np.zeros_like(y)
        for _ in range(self.n_estimators):
            residuals = y - current_estimate
            estimator = self._train_estimator(X, residuals)
            self.estimators_.append(estimator)
            current_estimate += estimator.predict(X)
        return self

    def _train_estimator(self, X, residuals):
        max_residual_idx = np.argmax(residuals)
        max_residual_object = X[max_residual_idx]
        
#         H = cpp_algorithm.find_irreducible_cover(max_residual_object) ToDo

        X_train = np.array([max_residual_object])
        y_train = np.array([residuals[max_residual_idx]])
        
        return SomeRegressor().fit(X_train, y_train)

    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)
        return sum(
            estimator.predict(X) for estimator in self.estimators_
        )

## Baseline

In [ ]:
# Генерация случайных данных для регрессии
X, y = make_regression(n_samples=100, n_features=10, noise=0.5, random_state=42)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели BoostingRegressor
model = BoostingRegressor(n_estimators=100)
model.fit(X_train, y_train)

# Предсказание значений
y_pred = model.predict(X_test)

# Вычисление среднеквадратичной ошибки
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Сравнение с простой моделью Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)
mse_linear = mean_squared_error(y_test, y_pred_linear)
print("Mean Squared Error (Linear Regression):", mse_linear)